# Import modules

In [1]:
import pandas as pd
import re
import urllib.request as urllib2
import time
from bs4 import BeautifulSoup, SoupStrainer


from tqdm import tqdm

# https://stackoverflow.com/questions/41529016/python-http-client-incomplete-read0-bytes-read-error
import http.client
http.client.HTTPConnection._http_vsn = 10
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

# Mount google drive

In [2]:
# References:
# [1] https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a
# [2] https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab
# [3] https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab

try:
    from google.colab import drive
    from google.colab import files

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:

    # mount google drive 
    drive.mount('/content/gdrive/', force_remount=True)

    import os
    #change directory
    try:
        os.chdir('/content/gdrive/MyDrive/ADC')

    except:
        os.chdir('/content/gdrive/MyDrive/ADC')

# Metainformation

In [3]:
dict_repository = {
       'UPorto'    : 'repositorio-aberto.up.pt', 
       'UCoimbra'  : 'estudogeral.sib.uc.pt',
       'ULisboa'   : 'repositorio.ul.pt',
       'UNLisboa'  : 'run.unl.pt', 
       'UMinho'    : 'repositorium.sdum.uminho.pt',
       'UAveiro'   : 'ria.ua.pt', 
       'UTL'       : 'www.repository.utl.pt', 
       'UCP'       : 'repositorio.ucp.pt',
       'ISCTE'     : 'repositorio.iscte-iul.pt',
       'IPPorto'   : 'recipp.ipp.pt', 
       'UBI'       : 'ubibliorum.ubi.pt',
       'UEvora'    : 'dspace.uevora.pt/rdpc',
       'UTAD'      : 'repositorio.utad.pt',
       'IPLisboa'  : 'repositorio.ipl.pt',
       'Lusófona'  : 'recil.ensinolusofona.pt',
       'UFPessoa'  : 'bdigital.ufp.pt',
       'UAlg'      : 'sapientia.ualg.pt',
       'ISPA'      : 'repositorio.ispa.pt',
       'IPCoimbra' : 'comum.rcaap.pt',
       'ICOnline'  : 'iconline.ipleiria.pt'
       }

In [4]:
###############################################################     UPorto
#
dict_UPorto = {
                'repository'    : 'repositorio-aberto.up.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.creator'],
                'Supervisor'    : ['dc.creator'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['dc.contributor.uporto'],
                'Degree'        : ['thesis.degree.name', 'thesis.degree.discipline'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UCoimbra
#
dict_UCoimbra = {
                'repository'    : 'estudogeral.sib.uc.pt',
                'classTable'    : 'itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['uc.degree.grantorUnit'],
                'Degree'        : ['thesis.degree.name'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
}

###############################################################     ULisboa
#
# Faculty and MSc program are included in dc.description
dict_ULisboa = {
                'repository'    : 'repositorio.ul.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['sdum.uoei'],                                # No example found
                'Degree'        : ['thesis.degree.name', 'dc.description'],     # Includes the MSc program and the Faculty 
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }


###############################################################     UNLisboa
#
dict_UNLisboa = {
                'repository'    : 'run.unl.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['sdum.uoei'],                                # No example found
                'Degree'        : ['thesis.degree.name', 'dc.description'],     # Includes the MSc program and the Faculty 
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UMinho
#
dict_UMinho = {
                'repository'    : 'repositorium.sdum.uminho.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['sdum.uoei'],
                'Degree'        : ['thesis.degree.name', 'thesis.degree.discipline'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UAveiro
#
dict_UAveiro = {
                'repository'    : 'ria.ua.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['sdum.uoei'],
                'Degree'        : ['thesis.degree.name', 'dc.description.master'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UTL
#
# MSc program is included in dc.description
dict_UTL = {
                'repository'    : 'www.repository.utl.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor', 'dc.contributor.other'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['dc.publisher'],
                'Degree'        : ['thesis.degree.name', 'dc.description'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     Católica
#
dict_UCP = {
                'repository'    : 'repositorio.ucp.pt',
                'classTable'    : 'panel-body table itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Language'      : ['dc.language.iso'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['sdum.uoei'],
                'Degree'        : ['thesis.degree.name'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UBI
#
dict_UBI = {
                'repository'    : 'ubibliorum.ubi.pt',
                'classTable'    : 'itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['dc.identifier.sharewith'],
                'Degree'        : ['thesis.degree.name'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UEvora
#
dict_UEvora = {
                'repository'    : 'dspace.uevora.pt/rdpc',
                'classTable'    : 'itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['dc.identifier.sharewith'],
                'Degree'        : ['thesis.degree.name'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }

###############################################################     UTAD
#
dict_UTAD = {
                'repository'    : 'repositorio.utad.pt',
                'classTable'    : 'itemDisplayTable',
                'Author'        : ['dc.contributor.author'],
                'Supervisor'    : ['dc.contributor.advisor'],
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Date'          : ['dc.date.issued'],
                'Abstract'      : ['dc.description.abstract'],
                'AbstractALT'   : ['dc.description.abstract'],
                'Language'      : ['dc.language.iso'],
                'Keywords'      : ['dc.subject'],
                'Faculty'       : ['uc.degree.grantorUnit'],
                'Degree'        : ['thesis.degree.name'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos'],
            }


###############################################################     UAlg
#
dict_UAlg = {
                'repository'    : 'sapientia.ualg.pt',
                'classTable'    : 'itemDisplayTable',
                'Author'        : ['dc.contributor.author'],               #
                'Supervisor'    : ['dc.contributor.advisor'],           #
                'Title'         : ['dc.title'],
                'TitleALT'      : ['dc.title.alternative'],
                'Language'      : ['dc.language.iso'],
                'Date'          : ['dc.date.issued'],           #
                'Abstract'      : ['dc.description.abstract'],         #
                'AbstractALT'   : ['dc.description.abstract'],         #
                'Keywords'      : ['dc.subject'],        #
                'Faculty'       : ['thesis.degree.grantor'],
                'Degree'        : ['thesis.degree.name'],
                'Type'          : ['dc.type'],
                'TID'           : ['dc.identifier.tid'],
                'Subject FOS'   : ['dc.subject.fos']
            }

In [5]:
dict_unis ={
            'UPorto'     : dict_UPorto,
            'UCoimbra'  : dict_UCoimbra,
            'ULisboa'   : dict_ULisboa,
            'UNLisboa'  : dict_UNLisboa, 
            'UMinho'     : dict_UMinho,
            'UAveiro'   : dict_UAveiro, 
            'UTL'       : dict_UTL, 
            'UCP'        : dict_UCP,
            # 'ISCTE'     : dict_ISCTE,
            # 'IPPorto'   : dict_IPPorto, 
            'UBI'       : dict_UBI,
            'UEvora'    : dict_UEvora,
            'UTAD'      : dict_UTAD,
            # 'IPLisboa'  : dict_IPLisboa,
            # 'Lusófona'  : dict_Lusófona,
            # 'UFPessoa'  : dict_UFPessoa,
            'UAlg'      : dict_UAlg,
            # 'ISPA'      : dict_ISPA,
            # 'IPCoimbra' : dict_IPCoimbra,
            # 'ICOnline'  : dict_ICOnline
       }

# Auxilliary functions

In [6]:
#################################################################################
#
def scrap_collection(soup):
    #
    # Scraps links to collections
    #

    # finds all links in the soup which have '/handle/' in the link
    l_link = soup.find_all(href=re.compile("/handle/"))
    
    l_out = []

    for l in l_link:
        # the links we want do not have class
        if len(re.findall(r'class="(.*?)"', str(l))) > 0:
            continue
        
        # if the link does not have class, append
        l_out.append((l.string, l['href']))

    # eliminates duplicates and cast to list
    return list(set(l_out))
    

#################################################################################
#
def save_files (df_MSTheses, l_rows, l_failures, fname, wkdir = None):
    #
    # Saves dataframes to csv files
    #

    # creates dataframe with new rows
    df = pd.DataFrame(l_rows)

    # concatenates to main dataframe
    df_MSTheses =  pd.concat([df_MSTheses, df]).reset_index().drop('index', axis = 1)

    # saves dataframes to csv files
    df_MSTheses.to_csv(wkdir + 'df_MSTheses_' + fname, index = False)
    df_failures = pd.DataFrame(l_failures)
    df_failures.to_csv(wkdir +'df_failures_' + fname, index = False)

    return df_MSTheses

# Repository scrapper

In [7]:
import copy

def repo_scrapper (university, dict_unis):

    print(f'Downloading {university}')

    dict_fields = copy.deepcopy(dict_unis[university])
    repository = dict_fields['repository']
    dict_fields.pop('repository')

    classTable = dict_fields['classTable']
    dict_fields.pop('classTable')

    l_columns = ['University', 'Page', 'URL'] + list(dict_fields.keys())

    dfLinks = pd.read_csv('DF_CORR/df_urls_' + university + '_corr.csv')
    dfLinks = dfLinks[dfLinks.urls != '[]']

    preLink = 'https://' + dict_repository[university] + '/handle/'

    MSTheses_dir = 'DF_MSTHESES/'

    # file name ending
    fname = university + '.csv'



    #################################################################################
    # If available, reads previously saved dataframes and continues the download
    try:
        df_MSTheses = pd.read_csv(MSTheses_dir + 'df_MSTheses_' + fname)
        idx_init = df_MSTheses['Page'].iloc[len(df_MSTheses) - 1]
        print(f'Continue from page {idx_init}')

        try:
            df_failures = pd.read_csv(MSTheses_dir + 'df_failures_' + fname)
            l_failures = df_failures['0'].to_list()
        except:
            l_failures = []

    except:
        df_MSTheses = pd.DataFrame(columns = l_columns)
        idx_init = 0
        l_failures = []
        print(f'Failed to open df_MSTheses_{fname}')



    # list of rows to append to df_MSTheses
    l_rows = []


    # for i in tqdm(range(idx_init, idx_init + 10)):
    for i in tqdm(range(idx_init, len(dfLinks))):
    
        # page where it was retrieved in RCAAP
        page = dfLinks.page[i]

        # list of links as string
        linksStr = dfLinks.urls[i]
        
        # list of links
        linksList = re.findall(r"'(.*?)'", linksStr)

        for j in range(len(linksList)):
            
            # link of the MSc dissertation
            link = linksList[j]
            link = link.split(':')[1]

            # link to hmtl page
            fullLink = preLink + link  + '?mode=full'

            # open webpage
            try:
                response = urllib2.urlopen(fullLink)
            except:
                l_failures.append(fullLink)
                continue

            readLink = response.read()

            # parse html file
            soup = BeautifulSoup(readLink, 'html.parser') #, from_encoding=response.info().get_param('charset'))

            # Extract table             
            # https://stackoverflow.com/questions/41100451/load-scraped-table-via-bs4-into-pandas-dataframe

            table =  soup.find('table', class_=classTable)
            # if the page is empty of tables
            if table == None:
                l_failures.append(fullLink)
                continue

            # extracts the table dataframe
            df = pd.read_html(table.prettify())[0]

            # dictionary with the values of each row
            dict_row = {
                        'University'    : university,
                        'Page'          : page,
                        'URL'           : fullLink,
                    }

            # loops all fields and adds to the dictionary
            for key in list(dict_fields.keys()):
                # content = df[df['DC Field'].isin(dict_fields[key])]['Value'].to_list()
                dict_row[key] = df[df.iloc[:,0].isin(dict_fields[key])].iloc[:,1].to_list()

            dict_row['Collection'] = scrap_collection(soup)
            # appends the dictionary to the list of rows
            l_rows.append(dict_row)


        # Creates backup
        if i % 10 == 0:

            # Save dataframes to files
            df_MSTheses = save_files (df_MSTheses, l_rows, l_failures, fname, MSTheses_dir)
            
            # resets the list
            l_rows = []

    # Save dataframes to files
    df_MSTheses = save_files (df_MSTheses, l_rows, l_failures, fname, MSTheses_dir)

    print(f'Number of {university} failures: {len(l_failures)}')

    return df_MSTheses, l_failures


# Universisties to download

In [8]:

#################################################################################
# List of universities to scrap
#
l_university = [
                'UPorto',
                # 'UCoimbra',
                # 'ULisboa',
                # 'UNLisboa', 
                # 'UMinho',
                # 'UAveiro', 
                # 'UTL', 
                # 'UCP',
                # 'ISCTE',
                # 'IPPorto', 
                # 'UBI',
                # 'UEvora',
                # 'UTAD',
                # 'IPLisboa',
                # 'Lusófona',
                # 'UFPessoa',
                # 'UAlg',
                # 'ISPA',
                # 'IPCoimbra',
                # 'ICOnline'
            ]
#
#################################################################################

for university in l_university:
    
    df_MSTheses, l_failures = repo_scrapper (university, dict_unis)

Continue from page 1565


100%|██████████| 1905/1905 [2:05:46<00:00,  3.96s/it]   


Number of UPorto failures: 82


In [9]:
df_MSTheses.iloc[1]['URL']

'https://repositorio-aberto.up.pt/handle/10216/142851?mode=full'

In [10]:
df_MSTheses

,Page,URL,Author,Supervisor,Title,TitleALT,Date,Abstract,AbstractALT,Language,Keywords,Faculty,Degree,Type,TID,Subject FOS,University,Collection
0,1,https://repositorio-aberto.up.pt/handle/10216/...,['Guilherme de Jesus Costa'],['Guilherme de Jesus Costa'],['A influência das estatinas no desenvolviment...,[],['2022-07-11'],[],[],['por'],"['Ciências médicas e da saúde', 'Medical and H...",['Faculdade de Medicina Dentária'],['Mestrado Integrado em Medicina Dentária'],['Dissertação'],['203046579'],"['Ciências médicas e da saúde', 'Medical and H...",UPorto,"[('FMDUP - Dissertação', '/handle/10216/6814')..."
1,1,https://repositorio-aberto.up.pt/handle/10216/...,['Leonor Torres dos Santos Hipólito Reis'],['Leonor Torres dos Santos Hipólito Reis'],['Sales and Operations Planning: aplicação de ...,[],['2022-07-22'],[],[],['por'],['Outras ciências da engenharia e tecnologias'...,['Faculdade de Engenharia'],['Mestrado em Engenharia e Gestão Industrial'],['Dissertação'],[],['Ciências da engenharia e tecnologias::Outras...,UPorto,"[('FEUP - Dissertação', '/handle/10216/6812'),..."
2,1,https://repositorio-aberto.up.pt/handle/10216/...,['Uindinéa Pedroza Almeida'],['Uindinéa Pedroza Almeida'],['Cidade educadora na contemporaneidade: a exp...,[],['2022-07-15'],['Desde o seu surgimento as cidades têm passad...,[],['por'],"['Ciências da educação', 'Educational sciences']",['Faculdade de Psicologia e de Ciências da Edu...,['Mestrado em Ciências da Educação'],['Dissertação'],['203049527'],"['Ciências sociais::Ciências da educação', 'So...",UPorto,"[('FPCEUP - Dissertação', '/handle/10216/6811'..."
3,1,https://repositorio-aberto.up.pt/handle/10216/...,['Rita Catarina da Silva Almeida de Carvalho'],['Rita Catarina da Silva Almeida de Carvalho'],['Relatório de Estágio - Serviço de Estomatolo...,[],['2022-07-12'],[],[],['por'],"['Ciências médicas e da saúde', 'Medical and H...",['Faculdade de Medicina Dentária'],['Mestrado Integrado em Medicina Dentária'],['Dissertação'],['203048156'],"['Ciências médicas e da saúde', 'Medical and H...",UPorto,"[('FMDUP - Dissertação', '/handle/10216/6814')..."
4,1,https://repositorio-aberto.up.pt/handle/10216/...,['Miguel Viamonte Santos da Costa Fernandes'],['Miguel Viamonte Santos da Costa Fernandes'],['Design de Uma App Planeadora de Rotas Multic...,[],['2022-10-07'],[],[],['por'],['Outras ciências da engenharia e tecnologias'...,['Faculdade de Engenharia'],['Mestrado em Engenharia e Gestão Industrial'],['Dissertação'],[],['Ciências da engenharia e tecnologias::Outras...,UPorto,"[('FEUP - Dissertação', '/handle/10216/6812'),..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34606,3470,https://repositorio-aberto.up.pt/handle/10216/...,"[Faria, Rui Manuel Carvalho Marques de]","[Faria, Rui Manuel Carvalho Marques de]",[Análise numérica de barragens abóbada sujeita...,[],[1988],[Dissertação apresentada para obtenção do grau...,[],[],"[Estruturas, Engenharia civil, Análise e métod...",[],[],[Dissertação],[],[],UPorto,"[(FEUP - Faculdade de Engenharia, /handle/1021..."
34607,3470,https://repositorio-aberto.up.pt/handle/10216/...,[António Pedro Oliveira de Carvalho],[António Pedro Oliveira de Carvalho],[Estudo acústico de divisórias amovíveis. Anál...,[],[1988],[],[],[por],"[Engenharia civil, Civil engineering]",[Faculdade de Engenharia],[],[Dissertação],[],[Ciências da engenharia e tecnologias::Engenha...,UPorto,"[(FEUP - Faculdade de Engenharia, /handle/1021..."
34608,3470,https://repositorio-aberto.up.pt/handle/10216/...,"[Lopes, Luis Manuel Bragança de Miranda e]","[Lopes, Luis Manuel Bragança de Miranda e]",[Cálculo de cargas térmicas de paredes exterio...,[],[1988],[Dissertação apresentada para obtenção do grau...,[],[],"[Engenharia térmica, Térmica de edifícios]",[],[],[Dissertação],[],[],UPorto,"[(FEUP - Faculdade de Engenharia, /handle/1021..."
34609,3470,https://repositorio-aberto.up.pt/handle/10216/...,"[Sousa, Jorge Rui Guimarães Freire de]","[Sousa, Jorge Rui Guimarães Freire 